## Simple Value Investing , using python (proof of concept)

This application refer [Vincent Tatan's blog at Medium](https://towardsdatascience.com/value-investing-dashboard-with-python-beautiful-soup-and-dash-python-43002f6a97ca)  to help user to makie profitable trading decision base on stock price history , financial summary , basic value analysis and infered reasonable share price. 

Most codes are based on Vincent's github project and do some modification in order to run in Jupyter notebook and  my prefered UI.  You can go to Vincent's blog to get complete explanation. 

### This application has not been rigorously tested and its domain rules are very simple which are hard to cover the complexity of real stock market. There may be also some bugs cause wrong information.
## <span style='color:blue'>Don't use this application to make real trading decision. </span>



------------------

### Main process :

1. Get stock ticker and name to be data source of dropdown control
2. Get stock price history from yahoo through pandas datareader when user choose a stock from dropdown control
3. Plot stock k chart , sma  , macd , rsi lines
4. Get stock financial information from financial website to do basic analysis and infer future value for trading decision.



### Parsing fincial information from MarketWatch web site
    For example, to get Google's financial information from this url : https://www.marketwatch.com/investing/stock/GOOG/financials



### Use pandas datareader through yahoo to get share price history
```
    pdr.DataReader(
            ticker.strip(),data_source='yahoo',start=dt(2020,1,1),end=end_date
    )
```


### Warning Signs List based on value investing logic ###

Given list of the companies, find out the feasibility to invest

1. Been in market minimal 10 years
1. Have the track records (EPS per year)
1. Have efficiency (ROE > 15%) — Net income / shareholder equity
1. Determine manipulation (ROA > 7%) — Net income / Total Asset
1. Have small long term debt (Long term debt <5* total income)
1. Low Debt to Equity
1. Ability to pay interest: (Interest Coverage Ratio >3) — EBIT / Interest expenses



### Decision rules for stock transaction

Decision making from each company in terms of return rate given the value investing methodology :

1. Find EPS Annual Compounded Growth Rate
2. Estimate EPS 10 years from now
3. Estimate stock price 10 years from now (Stock Price EPS * Average PE)
4. Determine target by price today based on returns(discount rate 15%/20%)
   - Discount rate indicated predicted inflation over the period of 5 years. 
5. Add margin of safety (Safety net 15%)
   - Margin calculation rate is the toleration for error. You add on additional safety net in the case of calculation error. The more the margin calculation rate, the more risk averse you are or vice versa.
6. Buy if market price is lower than the marginal price
7. Sell if market price is higher than the marginal price

In [27]:
import numpy as np
import numpy_financial as npf
import dash_bootstrap_components as dbc
import webbrowser
import dashboard 
from dashboard import *
import importlib
import logging
from flask.logging import default_handler
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
#%matplotlib inline
cf.offline.go_offline() # with execute this command will cause QuantFig.iplot() run into error

In [28]:


app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dashboard_layout()
app.title = 'Simple Value Investing Dashboard'
#logger = logging.getLogger()
#logger.addHandler(default_handler)
#logger.setLevel(logging.DEBUG)
#setLogger(logger)
register_callbacks(app)
# Run app and display result inline in the notebook
if __name__ == '__main__':
    app.run_server(
        debug=True,
        mode='external',    # inline  external  jupyterlab
        port=8123,
        dev_tools_ui=True,
        dev_tools_silence_routes_logging=False) 
    # set mode='external' to use web browser
    # dev_tools_silence_routes_logging=False
    url = 'http://localhost:8123/'
    webbrowser.open(url)
    


127.0.0.1 - - [09/Sep/2020 21:06:17] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 21:06:18] "GET /_shutdown_65a6c609-c8e9-4e8f-a541-83e64e675a1f HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 21:06:18] "GET /_alive_65a6c609-c8e9-4e8f-a541-83e64e675a1f HTTP/1.1" 200 -
Dash app running on http://127.0.0.1:8123/
 * Running on http://127.0.0.1:8123/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Sep/2020 21:06:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 21:06:19] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 21:06:19] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 21:06:20] "POST /_dash-update-component HTTP/1.1" 200 -
https://www.marketwatch.com/investing/stock/GOOG/financialshttps://www.marketwatch.com/investing/stock/GOOG/financialshttps://www.marketwatch.com/investing/stock/GOOG/financials


127.0.0.1 - - [09/Sep/2020 21:06:22] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 21:06:23] "POST /_dash-update-component HTTP

In [36]:
financial_report = get_stock_financial_report('GOOG')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.marketwatch.com:443
https://www.marketwatch.com/investing/stock/GOOG/financials
127.0.0.1 - - [09/Sep/2020 18:34:21] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:34:21] "GET /_reload-hash HTTP/1.1" 200 -
DEBUG:urllib3.connectionpool:https://www.marketwatch.com:443 "GET /investing/stock/GOOG/financials HTTP/1.1" 200 31609
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.marketwatch.com:443
127.0.0.1 - - [09/Sep/2020 18:34:24] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:34:24] "GET /_reload-hash HTTP/1.1" 200 -
DEBUG:urllib3.connectionpool:https://www.marketwatch.com:443 "GET /investing/stock/GOOG/financials/balance-sheet HTTP/1.1" 200 33513
127.0.0.1 - - [09/Sep/2020 18:34:27] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:34:27] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:34:30] "

In [37]:
financial_df =get_stock_financial_info_from_report(financial_report)

127.0.0.1 - - [09/Sep/2020 18:35:03] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:35:03] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:35:06] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:35:06] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:35:09] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:35:09] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:35:12] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:35:12] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:35:15] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:35:15] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:35:18] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:35:18] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:35:22] "GET /_reload-h

In [39]:
stockprice=get_stock_price('GOOG','2015-09-09',end_date=dt.now())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): finance.yahoo.com:443
127.0.0.1 - - [09/Sep/2020 18:36:26] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:36:26] "GET /_reload-hash HTTP/1.1" 200 -
DEBUG:urllib3.connectionpool:https://finance.yahoo.com:443 "GET /quote/GOOG/history?period1=1577822400&period2=1599681599&interval=1d&frequency=1d&filter=history HTTP/1.1" 200 None
127.0.0.1 - - [09/Sep/2020 18:36:30] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:36:30] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:36:34] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:36:34] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:36:39] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:36:39] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2020 18:36:42] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [

In [50]:
infer_reasonable_share_price('GOOG',financial_df,stockprice,0.01,0.01)

127.0.0.1 - - [09/Sep/2020 18:57:24] "GET /_reload-hash HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2020 18:57:24] "GET /_reload-hash HTTP/1.1" 200 -
1st year EPS 23.120000 
5th year EPS 49.590000
Annual Growth Rate 0.164880
PE ration 26.961485


,annualgrowthrate,lasteps,futureeps,peratio,FV,PV,marginprice,lastshareprice,decision
ticker,,,,,,,,,
GOOG,0.16,49.59,228.14,26.96,"6,151.07","5,568.48","5,512.80","1,532.39",BUY


In [20]:
query_financial_report('GOOG')

https://www.marketwatch.com/investing/stock/GOOG/financials
                      eps            epsgrowth    netincome  \
2015                23.12                 0.00  15830000000   
2016                28.32                 0.23  19480000000   
2017                18.27                -0.35  12660000000   
2018                44.22                 1.42  30740000000   
2019                49.59                 0.12  34340000000   

      shareholderequity                  roa  longtermdebt  interestexpense  \
2015       120330000000                 0.82    2000000000        104000000   
2016       139040000000                 0.83    3940000000        124000000   
2017       152500000000                 0.77    3970000000        109000000   
2018       177630000000                 0.76    4010000000        114000000   
2019       201440000000                 0.73   14770000000        100000000   

           ebitda                  roe  interestcoverageratio  
2015  23020000000     

{'financial_report':         eps            epsgrowth netincome shareholderequity  \
 2015  23.12                  nan    15.83B           120.33B   
 2016  28.32                  nan    19.48B           139.04B   
 2017  18.27                  nan    12.66B            152.5B   
 2018  44.22                  nan    30.74B           177.63B   
 2019  49.59                  nan    34.34B           201.44B   
 
                       roa longtermdebt interestexpense  ebitda  \
 2015                  nan           2B            104M  23.02B   
 2016                  nan        3.94B            124M  29.32B   
 2017                  nan        3.97B            109M  35.97B   
 2018                  nan        4.01B            114M  40.57B   
 2019                  nan       14.77B            100M  47.25B   
 
                       roe  interestcoverageratio  
 2015                  nan                    nan  
 2016                  nan                    nan  
 2017                  nan  

In [18]:
 def query_financial_report(ticker):

        stock_financial_df = {} #pd.DataFrame()
        financial_report = get_stock_financial_report(ticker).reset_index()
        financial_info_df = get_stock_financial_info_from_report(financial_report).reset_index()
        print("financial_info_df",financial_info_df)
        financial_report[['roe', 'interestcoverageratio', 'epsgrowth', 'roa']] = np.round(
                    financial_info_df[['roe', 'interestcoverageratio', 'epsgrowth', 'roa']], 2)
        #print(stock_financial_df)
        stock_financial_df["financial_report"]=financial_report
        #print(stock_financial_df)
        stock_financial_df["financial_info_df"]=financial_info_df
        #print(stock_financial_df)
        #stock_financial_df_dict[ticker]=stock_financial_df
        return stock_financial_df

In [10]:
stock_financial_df = stock_financial_df_dict.get('GOOG')

In [11]:
stock_financial_df

{'financial_report':    index    eps            epsgrowth netincome shareholderequity  \
 0   2015  23.12                 0.00    15.83B           120.33B   
 1   2016  28.32                 0.23    19.48B           139.04B   
 2   2017  18.27                -0.35    12.66B            152.5B   
 3   2018  44.22                 1.42    30.74B           177.63B   
 4   2019  49.59                 0.12    34.34B           201.44B   
 
                    roa longtermdebt interestexpense  ebitda  \
 0                 0.82           2B            104M  23.02B   
 1                 0.83        3.94B            124M  29.32B   
 2                 0.77        3.97B            109M  35.97B   
 3                 0.76        4.01B            114M  40.57B   
 4                 0.73       14.77B            100M  47.25B   
 
                    roe  interestcoverageratio  
 0                 0.13                 221.35  
 1                 0.14                 236.45  
 2                 0.08        

In [29]:
del stock_financial_df_dict['GOOG']